In [1]:
from text_processing import get_text_map

/Users/lilyakhoang/anaconda3/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
берем первый текст,смотрим на следующий: если менее 10% лемм овпадает то принимаем его

In [3]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
import json

texts_lenta = pd.read_csv("./colloc/music_lenta.csv")

In [ ]:
def lemm_text(text, model):
    conll = get_conllu_from_unite_line_text(text, model)
    conllu_text_map = get_conllu_text_map(conll)
    lemm, sent = get_lemm_and_orig_text_from_udmap(conllu_text_map)
    lemm_list = []
    for sent in lemm:
        for word in sent.split():
            lemm_list.append(word)
    lemm_list = set(lemm_list)
    return lemm_list
lemm_list_text = lemm_text(texts_lenta['text'][0], model)

In [6]:
text_map_test = get_text_map(texts_lenta['text'][0], raw_text_input=True)

In [7]:
def get_text_prop(text_map):
    text_vector = []
    text_vector.append(text_map['lix'])
    text_vector.append(text_map['ttr'])
    text_vector.extend(text_map['sent_properties'])
    lemmas = []
    sentence_count = 0
    for sentence in text_map['sentences_map']:
        sentence_count += 1
        for word_el in sentence['sentence_words']:
            if not word_el['lemma'].isdigit():
                lemmas.append(word_el['lemma'])      
    lemmas = set(lemmas)
    
    #print(text_vector)
    #print(lemmas)
    return text_vector, lemmas, sentence_count
teest_text_vector_1, test_lemmas_1, sentence_1 = get_text_prop(text_map_test)

In [8]:
sentence_1

10

In [10]:
def get_text_prop_grom_raw_text(text):
    text_map = get_text_map(text, raw_text_input=True)
    text_vector, lemmas, sentence_count = get_text_prop(text_map)
    return text_vector, lemmas, sentence_count
teest_text_vector_2, test_lemmas_2,sentence_count = get_text_prop_grom_raw_text(texts_lenta['text'][12])

In [15]:
collected_texts = []
collected_texts.append({"text_index":0,"text_vector":teest_text_vector_1, "vocab":test_lemmas_1 })
for text_ind in range(1, len(texts_lenta)):#len(texts_lenta)
    print(text_ind)
    text_vector, candidat_lemmas_set, sentence_count= get_text_prop_grom_raw_text(texts_lenta['text'][text_ind])
    candidat_text_vector = np.array(text_vector).reshape(1,-1)
    distances = []
    intersections = []
    print("TOTAL SENT", sentence_count)
    if sentence_count < 30 and sentence_count > 12:
        for approved_text_prop in collected_texts:
            #print(approved_text_prop['text_index'])
            approved_text_vector = approved_text_prop['text_vector']
            approved_text_vector = np.array(approved_text_vector).reshape(1,-1)
            distance = cosine_similarity(candidat_text_vector, approved_text_vector)
            #print(distance[0][0])
            distances.append(distance[0][0])

            approved_lemmas_set = approved_text_prop['vocab']
            intersect = candidat_lemmas_set.intersection(approved_lemmas_set)
            intersection = len(intersect) / len(candidat_lemmas_set)
            #print(intersection)
            intersections.append(intersection)
        mean_distance = sum(distances) / len(distances)
        mean_intersections = sum(intersections) / len(intersections)
        print("MEAN",mean_distance,mean_intersections)
        if mean_distance < 0.95 and mean_intersections < 0.15:
            print(text_ind, "has been added")
            collected_texts.append({"text_index":text_ind, "text_vector":text_vector, "vocab":candidat_lemmas_set})

        if len(collected_texts) >= 15:
            break

1
TOTAL SENT 9
2
TOTAL SENT 6
3
TOTAL SENT 11
4
TOTAL SENT 15
MEAN 0.9909455202621045 0.16037735849056603
5
TOTAL SENT 9
6
TOTAL SENT 6
7
TOTAL SENT 14
MEAN 0.9775953140454093 0.15328467153284672
8
TOTAL SENT 13
MEAN 0.9707766572452207 0.12318840579710146
9
TOTAL SENT 7
10
TOTAL SENT 15
MEAN 0.9943954706294919 0.1232876712328767
11
TOTAL SENT 13
MEAN 0.9873310230472496 0.2421875
12
TOTAL SENT 10
13
TOTAL SENT 6
14
TOTAL SENT 12
15
TOTAL SENT 14
MEAN 0.977996425698644 0.087248322147651
16
TOTAL SENT 3
17
TOTAL SENT 14
MEAN 0.948291436721399 0.12101910828025478
17 has been added
18
TOTAL SENT 14
MEAN 0.9439258506974415 0.15838509316770186
19
TOTAL SENT 7
20
TOTAL SENT 14
MEAN 0.9526222404779411 0.1623931623931624
21
TOTAL SENT 7
22
TOTAL SENT 14
MEAN 0.9672929126206136 0.09191176470588236
23
TOTAL SENT 19
MEAN 0.9691179567996545 0.15
24
TOTAL SENT 12
25
TOTAL SENT 9
26
TOTAL SENT 15
MEAN 0.9463409880715081 0.13271604938271603
26 has been added
27
TOTAL SENT 14
MEAN 0.9635005081237676 0.1

TOTAL SENT 12
245
TOTAL SENT 12
246
TOTAL SENT 13
MEAN 0.9581185424469925 0.17845117845117847
247
TOTAL SENT 11
248
TOTAL SENT 17
MEAN 0.9592041058913454 0.12132822477650063
249
TOTAL SENT 98
250
TOTAL SENT 9
251
TOTAL SENT 12
252
TOTAL SENT 7
253
TOTAL SENT 17
MEAN 0.9610477153122952 0.10135135135135134
254
TOTAL SENT 9
255
TOTAL SENT 16
MEAN 0.9536695457473487 0.12178279974890144
256
TOTAL SENT 8
257
TOTAL SENT 38
258
TOTAL SENT 12
259
TOTAL SENT 19
MEAN 0.9480066096238262 0.16466826538768986
260
TOTAL SENT 6
261
TOTAL SENT 23
MEAN 0.9641340177658032 0.18469462840323767
262
TOTAL SENT 7
263
TOTAL SENT 11
264
TOTAL SENT 210
265
TOTAL SENT 7
266
TOTAL SENT 10
267
TOTAL SENT 11
268
TOTAL SENT 10
269
TOTAL SENT 15
MEAN 0.9605406879454605 0.15398075240594924
270
TOTAL SENT 19
MEAN 0.9442582838459997 0.16133082234777152
271
TOTAL SENT 12
272
TOTAL SENT 8
273
TOTAL SENT 11
274
TOTAL SENT 13
MEAN 0.9648370579915109 0.15415415415415418
275
TOTAL SENT 8
276
TOTAL SENT 22
MEAN 0.960611207410889

In [12]:
for text in collected_texts:
    text_ind = text['text_index']
    print(text_ind)


0
17
26
71
73
82
84
108
121
130
140
153
155
168
169


In [16]:
for text in collected_texts:
    text_ind = text['text_index']
    print(text_ind)
    print(texts_lenta['text'][text_ind])
    print(text['text_vector'])
    print()

0
Расходы на проведение музыкального конкурса «Евровидение-2017» в Киеве оказались предельно высокими. Об этом в четверг, 11 мая, пишет украинское издание «Вести». «Украинцев возмутили запредельные гонорары артистов и ведущих шоу, то, что почти все тендеры проводились в сжатые сроки по сокращенной процедуре, а также явно лишние, неоправданные траты в стране, где идет война (речь идет о вооруженном конфликте в Донбассе — прим. «Ленты.ру», а большой части населения не хватает денег даже на оплату коммуналки», — говорится в заметке. По подсчетам газеты, на соревнование было потрачено 870 миллионов гривен (33 миллиона долларов). «Ровно столько закладывали в проект госбюджета Украины 2017 года на дотации государственными угольным шахтам (частичное покрытие разницы в себестоимости угля)», — пишет издание. По мнению авторов, пять миллионов гривен (190 тысяч долларов), потраченные на белое полотно, которым задрапировали «недострой рядом с местом проведения конкурса», «явно можно было бы потрат

In [42]:
collected_texts[0]

{'text_index': 0,
 'text_vector': [0.6118,
  0.75,
  0.2,
  0.2,
  0.024266827607358696,
  0.03000229547540447,
  0.0,
  0.07131167644959036,
  0.191510992676087],
 'vocab': {'а',
  'автор',
  'артист',
  'белый',
  'больший',
  'большой',
  'бы',
  'быть',
  'в',
  'ведущий',
  'вести',
  'весь',
  'возмущать',
  'война',
  'вооружать',
  'второй',
  'высокий',
  'газета',
  'где',
  'говорить',
  'год',
  'гонорар',
  'госбюджет',
  'государственный',
  'гривна',
  'даже',
  'деньги',
  'дождь',
  'доллар',
  'донбасс',
  'дотация',
  'евровидение',
  'задрапировать',
  'закладывать',
  'заметка',
  'запредельный',
  'и',
  'идти',
  'издание',
  'киев',
  'коммуналки',
  'конкурс',
  'конкурса',
  'конфликт',
  'который',
  'лентыру',
  'лишний',
  'май',
  'место',
  'миллион',
  'мнение',
  'можно',
  'музыкальный',
  'на',
  'намечать',
  'население',
  'не',
  'недостра',
  'неоправданный',
  'непригодный',
  'о',
  'оказываться',
  'оплата',
  'отмечать',
  'первый',
  'писать'

In [32]:
teest_text_vector_2, test_lemmas_2 

([0.628,
  0.7,
  0.1,
  0.2,
  0.018383861086364594,
  0.013862954436335007,
  0.008031559732857345,
  0.0959318616997087,
  0.20981930707596907],
 {'lifenews',
  'александр',
  'аншлаговый',
  'арена',
  'артист',
  'белоруссия',
  'белорусский',
  'букет',
  'быть',
  'в',
  'вильнюс',
  'возможно',
  'впервые',
  'входить',
  'высокий',
  'выступать',
  'где',
  'граница',
  'давать',
  'даже',
  'дальнейший',
  'дворец',
  'декорация',
  'десятитысячный',
  'для',
  'добавлять',
  'документация',
  'должен',
  'драгон',
  'задерживать',
  'и',
  'изза',
  'изучать',
  'калининград',
  'каунася',
  'киркоров',
  'клайпед',
  'количество',
  'концерт',
  'корреспондент',
  'который',
  'кремлевский',
  'литва',
  'лукашенко',
  'март',
  'место',
  'минск',
  'москва',
  'мы',
  'на',
  'название',
  'накладный',
  'не',
  'ни',
  'новостей',
  'новый',
  'оборудование',
  'огромный',
  'один',
  'он',
  'она',
  'отказываться',
  'отправляться',
  'ошеломительный',
  'певец',
  'пе

In [28]:
teest_text_vector_1 

[0.6118,
 0.75,
 0.2,
 0.2,
 0.024266827607358696,
 0.03000229547540447,
 0.0,
 0.07131167644959036,
 0.191510992676087]

In [5]:
text_map

OrderedDict([('lix', 0.6118),
             ('ttr', 0.75),
             ('overall_colloc_text',
              [{0: [(0, 'расход на'),
                 (2, 'проведение'),
                 (3, 'музыкальный'),
                 (4, 'конкурс'),
                 (6, '2017'),
                 (7, 'в'),
                 (8, 'киев'),
                 (9, 'оказываться'),
                 (10, 'предельно'),
                 (11, 'высокий')]},
               {1: [(0, 'о это в четверг'),
                 (4, '11'),
                 (5, 'май'),
                 (6, 'писать'),
                 (7, 'украинский'),
                 (8, 'издание'),
                 (9, 'вести')]},
               {2: [(0, 'украинец'),
                 (3, 'гонорар'),
                 (4, 'артист'),
                 (5, 'и'),
                 (6, 'ведущий'),
                 (7, 'шоу'),
                 (8, 'то что'),
                 (10, 'почти'),
                 (11, 'весь'),
                 (12, 'тендер'),
           